# asyncio

## 快速上手

In [5]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
async def func():
    print("协程内部代码")
# 调用协程函数，返回一个协程对象。
result = func()
# 方式一
# loop = asyncio.get_event_loop() # 创建一个事件循环
# loop.run_until_complete(result) # 将协程当做任务提交到事件循环的任务列表中，协程执行完成之后终止。
# 方式二
# 本质上方式一是一样的，内部先 创建事件循环 然后执行 run_until_complete，一个简便的写法。
# asyncio.run 函数在 Python 3.7 中加入 asyncio 模块，
asyncio.run(result)

协程内部代码


## await

- 例1

In [6]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
async def func():
    print("执行协程函数内部代码")
    # 遇到IO操作挂起当前协程（任务），等IO操作完成之后再继续往下执行。
    # 当前协程挂起时，事件循环可以去执行其他协程（任务）。
    response = await asyncio.sleep(2)
    print("IO请求结束，结果为：", response)
result = func()
asyncio.run(result)

执行协程函数内部代码
IO请求结束，结果为： None


- 例2

In [8]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
async def others():
    print("start")
    await asyncio.sleep(2)
    print('end')
    return '返回值'
async def func():
    print("执行协程函数内部代码")
    # 遇到IO操作挂起当前协程（任务），等IO操作完成之后再继续往下执行。当前协程挂起时，事件循环可以去执行其他协程（任务）。
    response = await others()
    print("IO请求结束，结果为：", response)
asyncio.run( func() )

执行协程函数内部代码
start
end
IO请求结束，结果为： 返回值


- 例3

In [9]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def others():
    print("start")
    await asyncio.sleep(2)
    print('end')
    return '返回值'

async def func():
    print("执行协程函数内部代码")
    # 遇到IO操作挂起当前协程（任务），等IO操作完成之后再继续往下执行。当前协程挂起时，事件循环可以去执行其他协程（任务）。
    response1 = await others()
    print("IO请求结束，结果为：", response1)
    response2 = await others()
    print("IO请求结束，结果为：", response2)
    
asyncio.run( func() )

执行协程函数内部代码
start
end
IO请求结束，结果为： 返回值
start
end
IO请求结束，结果为： 返回值


## task对象

- 例1

In [11]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
async def func(num):
    print(num)
    await asyncio.sleep(2)
    print(num * 10 + num)
    return "返回值" + str(num) + '\n'
async def main():
    print("main开始")
    # 创建协程，将协程封装到一个Task对象中并立即添加到事件循环的任务列表中，等待事件循环去执行（默认是就绪状态）。
    task1 = asyncio.create_task(func(1))
    # 创建协程，将协程封装到一个Task对象中并立即添加到事件循环的任务列表中，等待事件循环去执行（默认是就绪状态）。
    task2 = asyncio.create_task(func(2))
    print("main结束")
    # 当执行某协程遇到IO操作时，会自动化切换执行其他任务。
    # 此处的await是等待相对应的协程全都执行完毕并获取结果
    ret1 = await task1
    ret2 = await task2
    print(ret1, ret2)
asyncio.run(main())

main开始
main结束
1
2
11
22
返回值1
 返回值2



- 例2

In [19]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
async def func():
    print(1)
    await asyncio.sleep(2)
    print(2)
    return "返回值"
async def main():
    print("main开始")
    # 创建协程，将协程封装到Task对象中并添加到事件循环的任务列表中，等待事件循环去执行（默认是就绪状态）。
    # 在调用
    task_list = [
        asyncio.create_task(func()),
        asyncio.create_task(func())
    ]
    print("main结束")
    # 当执行某协程遇到IO操作时，会自动化切换执行其他任务。
    # 此处的await是等待所有协程执行完毕，并将所有协程的返回值保存到done
    # 如果设置了timeout值，则意味着此处最多等待的秒，完成的协程返回值写入到done中，未完成则写到pending中。
    done, pending = await asyncio.wait(task_list, timeout=None)
    print(done, pending)
asyncio.run(main())

main开始
main结束
1
1
2
2
{<Task finished coro=<func() done, defined at <ipython-input-19-4992b676d4cf>:5> result='返回值'>, <Task finished coro=<func() done, defined at <ipython-input-19-4992b676d4cf>:5> result='返回值'>} set()


## Future

In [20]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def set_after(fut):
    await asyncio.sleep(2)
    fut.set_result("666")
async def main():
    # 获取当前事件循环
    loop = asyncio.get_running_loop()
    # 创建一个任务（Future对象），没绑定任何行为，则这个任务永远不知道什么时候结束。
    fut = loop.create_future()
    # 创建一个任务（Task对象），绑定了set_after函数，函数内部在2s之后，会给fut赋值。
    # 即手动设置future任务的最终结果，那么fut就可以结束了。
    await loop.create_task(set_after(fut))
    # 等待 Future对象获取 最终结果，否则一直等下去
    data = await fut
    print(data)
asyncio.run(main())

666


## concurrent.futures.Future对象

In [21]:
import time
from concurrent.futures import Future
from concurrent.futures.thread import ThreadPoolExecutor
from concurrent.futures.process import ProcessPoolExecutor
def func(value):
    time.sleep(1)
    print(value)
pool = ThreadPoolExecutor(max_workers=5)
# 或 pool = ProcessPoolExecutor(max_workers=5)
for i in range(10):
    fut = pool.submit(func, i)
    print(fut)

<Future at 0x13ac385a488 state=running>
<Future at 0x13ac37adec8 state=running>
<Future at 0x13ac3909508 state=running>
<Future at 0x13ac3acc288 state=running>
<Future at 0x13ac418b5c8 state=running>
<Future at 0x13ac3fca648 state=pending>
<Future at 0x13ac3fcae88 state=pending>
<Future at 0x13ac3fcaec8 state=pending>
<Future at 0x13ac3fca188 state=pending>
<Future at 0x13ac3614f88 state=pending>
10

234


5
9786



